In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = [
    'index_sum_events',
    'index_mean_events',
    'index_min_events',
    'index_max_events',
    'application_id_mean_events',
    'device_model_std_events',
    'kind_sum_events',
    'kind_mean_events',
    'kind_min_events',
    'ip_address_sum_events',
    'ip_address_mean_events',
    'ip_address_min_events',
    'ip_address_max_events',
    'hour_std_events',
    'application_id_sum_installs',
    'application_id_mean_installs',
    'application_id_min_installs',
    'application_id_max_installs',
    'ref_type_sum_installs',
    'ref_type_mean_installs',
    'ref_type_min_installs',
    'ref_type_max_installs',
    'device_countrycode_sum_installs',
    'device_countrycode_mean_installs',
    'device_countrycode_min_installs',
    'device_countrycode_max_installs',
    'ip_address_sum_installs',
    'ip_address_mean_installs',
    'ip_address_min_installs',
    'ip_address_max_installs',
    'repeticiones_sum_installs',
    'repeticiones_mean_installs',
    'repeticiones_min_installs',
    'repeticiones_max_installs',
    'device_countrycode6287817205707153877_sum_installs',
    'device_countrycode6287817205707153877_mean_installs',
    'device_countrycode6287817205707153877_min_installs',
    'device_countrycode6287817205707153877_max_installs',
    'hour_sum_auctions',
    'hour_mean_auctions',
    'hour_std_auctions',
    'hour_max_auctions',
    'source_id1_sum_auctions',
    'source_id1_mean_auctions',
    'source_id1_std_auctions',
    'hour3_std_auctions',
    'hour2_std_auctions',
    'hour1_std_auctions',
    'hour23_mean_auctions',
    'day2019-04-18_mean_auctions',
    'tiempo_hasta_aparicion_ventana',
    'time_diff_sum',
    'time_diff_mean',
    'time_diff_std',
    'time_diff_max',
    'time_diff_min']

In [3]:
drop_columns = [
    'index_sum_events',
    'application_id_mean_events',
    'kind_sum_events',
    'kind_mean_events',
    'ip_address_sum_events',
    'ip_address_mean_events',
    'application_id_sum_installs',
    'application_id_mean_installs',
    'ref_type_sum_installs',
    'device_countrycode_sum_installs',
    'device_countrycode_mean_installs',
    'device_countrycode_min_installs',
    'device_countrycode_max_installs',
    'ip_address_sum_installs',
    'ip_address_mean_installs',
    'device_countrycode6287817205707153877_sum_installs',
    'device_countrycode6287817205707153877_mean_installs',
    'device_countrycode6287817205707153877_min_installs',
    'device_countrycode6287817205707153877_max_installs',
    'hour_sum_auctions']

In [4]:
fill0_columns = [
    'index_mean_events',
    'ref_type_mean_installs',
    'repeticiones_sum_installs',
    'repeticiones_mean_installs',
    'hour_mean_auctions',
    'source_id1_sum_auctions',
    'source_id1_mean_auctions',
    'day2019-04-18_mean_auctions']

In [5]:
fill_max_columns = [
    'tiempo_hasta_aparicion_ventana',
    'time_diff_sum',
    'time_diff_mean',
    'time_diff_std',
    'time_diff_max',
    'time_diff_min']

In [6]:
features_ventana1 = pd.read_csv('../xgb/features_clicks_gonzalo_sc_ventana1.csv').set_index('ref_hash').add_suffix('_clicks')
aux_ventana1 = pd.read_csv('../xgb/features_events_gonzalo_sc_ventana1.csv').set_index('ref_hash').add_suffix('_events')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer')
aux_ventana1 = pd.read_csv('../xgb/features_installs_gonzalo_sc_ventana1.csv').set_index('ref_hash').add_suffix('_installs')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer')
aux_ventana1 = pd.read_csv('../xgb/features_auctions_gonzalo_sc_ventana1.csv').set_index('ref_hash').add_suffix('_auctions')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer')
aux_ventana1 = pd.read_csv('../xgb/features_sc_ventana1.csv').set_index('ref_hash').add_suffix('_ventana')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer')
aux_ventana1 = pd.read_csv('../xgb/features_timediff_installs_ventana1.csv').set_index('ref_hash')
features_ventana1 = features_ventana1.join(aux_ventana1, how = 'outer')

features_ventana1.drop(drop_columns, axis=1, inplace = True)

for i in fill_max_columns:
    features_ventana1[i] = features_ventana1[i].fillna(259200)
for i in fill0_columns:
    features_ventana1[i] = features_ventana1[i].fillna(0)

In [7]:
labels_sc_ventana1 = pd.read_csv('../xgb/labels_sc_ventana1.csv').set_index('ref_hash').add_suffix('_target')
ventana1 = features_ventana1.join(labels_sc_ventana1, how = 'left')
ventana1['tiempo_hasta_aparicion_target'] = ventana1['tiempo_hasta_aparicion_target'].fillna(259200)#3 dias pasado a segundos
ventana1_hit = ventana1.loc[ventana1['tiempo_hasta_aparicion_target'] < 259200]
ventana1_no_hit = ventana1.loc[ventana1['tiempo_hasta_aparicion_target'] == 259200]
registros_hit = float(ventana1_hit.shape[0])
registros_no_hit = ventana1_no_hit.shape[0]
relacion_deseada_hit_a_no_hit = 4
fraccion = registros_hit / (relacion_deseada_hit_a_no_hit * registros_no_hit)
ventana1_no_hit = ventana1_no_hit.sample(frac= fraccion)
ventana1 = ventana1_hit.append(ventana1_no_hit)

In [8]:
matriz = ventana1
matriz.to_csv('../xgb/training_set_sc.csv')